In [41]:
# First Conclusions of Unsupervised data analisys
# Data not Normally distributed ? 


# make modules autoreloading
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from copy import copy

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')
    
from zaCode import unsupervised
from zaCode import FileManager
from zaCode import DatasetManipulator
from zaCode.DatasetManipulator import DSetTransform
from zaCode.unsupervised import ProbEngine
from zaCode import Validator

dset_trans = DSetTransform()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%reload_ext autoreload
%autoreload


data_test = FileManager.getTestData()
dtA = dset_trans.periodic_partition(data_test, 0.3)

selecting 102329 items from data
done with iteration number 20000
done with iteration number 40000
done with iteration number 60000
done with iteration number 80000
done with iteration number 100000
done with iteration number 120000
done with iteration number 140000
done with iteration number 160000
done with iteration number 180000
done with iteration number 200000
done with iteration number 220000
done with iteration number 240000
done with iteration number 260000
done with iteration number 280000
done with iteration number 300000
done with iteration number 320000
done with iteration number 340000


In [6]:
dtA.to_csv("../data/orders_train_subsample.csv")

In [95]:

data = pd.read_csv('../data/orders_train_condproba.csv')
data_first10k = copy(data[0:20000])
data_second10k = copy(data[20000:40000])

In [96]:
data_first10k = data_first10k.drop(data_first10k.columns[0], axis = 1)
data_second10k = data_second10k.drop(data_second10k.columns[0], axis = 1)

In [97]:
data_first10k.to_csv('../data/orders_train_condproba_first20k.csv')
data_second10k.to_csv('../data/orders_train_condprobs_second20k.csv')

In [98]:

data_first10k = data_first10k.drop(['customerID', 'orderID', 'orderDate'], axis = 1)
data_second10k = data_second10k.drop(['customerID', 'orderID', 'orderDate'], axis = 1)

In [99]:
sup_c = 'returnQuantity'
feats_c = copy(data_first10k.columns).drop(sup_c)
feats_c

Xunsup = data_second10k.drop(sup_c, axis = 1)
Ytest  = copy(data_second10k[sup_c])

Xtrain = data_first10k.drop(sup_c, axis = 1)
Ytrain = copy(data_first10k[sup_c])

In [100]:
# train without unsupervised features
from sklearn.ensemble import GradientBoostingClassifier

baseline = GradientBoostingClassifier(verbose=1, max_depth=2, learning_rate=0.1, n_estimators = 800)
baseline.fit(Xtrain, Ytrain)
Ypred = baseline.predict(Xunsup)
Validator.performValidation(Ypred, Ytest)


      Iter       Train Loss   Remaining Time 
         1           1.3654            3.09m
         2           1.3551            1.65m
         3           1.3464            1.17m
         4           1.3385           56.44s
         5           1.3317           47.58s
         6           1.3259           41.12s
         7           1.3209           36.37s
         8           1.3165           32.86s
         9           1.3125           30.06s
        10           1.3088           27.84s
        20           1.2869           17.92s
        30           1.2773           14.41s
        40           1.2723           12.62s
        50           1.2692           11.47s
        60           1.2670           10.60s
        70           1.2654            9.98s
        80           1.2640            9.48s
        90           1.2627            9.05s
       100           1.2617            8.68s
       200           1.2541            6.81s
       300           1.2483            5.60s
       40

In [101]:
pb_eng = ProbEngine(Xunsup)
gb_cv = pb_eng.global_cov()

Xunsup_cv = copy(Xunsup)
Xtrain_cv = copy(Xtrain)

Xunsup_cv['gb_Mhl'] = gb_cv.mahalanobis(Xunsup_cv)
Xtrain_cv['gb_Mhl'] = gb_cv.mahalanobis(Xtrain_cv)

In [102]:

baseline = GradientBoostingClassifier(verbose=1, max_depth=2, learning_rate=0.1, n_estimators = 800)
baseline.fit(Xtrain_cv, Ytrain)
Ypred = baseline.predict(Xunsup_cv)
Validator.performValidation(Ypred, Ytest)

      Iter       Train Loss   Remaining Time 
         1           1.3654           12.06s
         2           1.3551           12.05s
         3           1.3464           11.40s
         4           1.3385           11.20s
         5           1.3317           10.96s
         6           1.3259           10.78s
         7           1.3209           10.55s
         8           1.3165           11.12s
         9           1.3125           11.90s
        10           1.3088           11.63s
        20           1.2869           10.74s
        30           1.2773            9.98s
        40           1.2723            9.69s
        50           1.2689            9.30s
        60           1.2667            9.09s
        70           1.2648            9.40s
        80           1.2633            9.65s
        90           1.2621            9.45s
       100           1.2610            9.28s
       200           1.2517            7.60s
       300           1.2446            6.04s
       40

In [31]:


data_second10k

,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,deviceID,paymentMethod,returnQuantity
10001,0.561462,0.513603,0.524552,0.461399,1,29.99,29.99,0.519868,0.0,0.519731,0.371715,0
10002,0.640599,0.447977,0.512356,0.582717,1,35.00,49.99,0.519868,0.0,0.509687,0.555840,1
10003,0.657925,0.476686,0.512356,0.582717,1,45.99,45.99,0.519868,0.0,0.509687,0.555840,1
10004,0.508547,0.506116,0.372420,0.378380,1,27.99,27.99,0.519868,0.0,0.519731,0.555840,0
10005,0.489933,0.535307,0.530262,0.461399,1,20.00,39.99,0.553106,10.0,0.519731,0.555840,1
10006,0.489933,0.535307,0.524552,0.461399,1,20.00,39.99,0.553106,10.0,0.519731,0.555840,1
10007,0.304348,0.535307,0.530262,0.461399,2,50.00,29.99,0.553106,10.0,0.519731,0.555840,1
10008,0.463519,0.429368,0.530262,0.461399,2,40.00,39.99,0.553106,10.0,0.519731,0.555840,1
10009,0.463519,0.429368,0.524552,0.461399,1,20.00,39.99,0.553106,10.0,0.519731,0.555840,1
10010,0.411765,0.408163,0.530262,0.461399,1,20.00,29.99,0.553106,10.0,0.519731,0.555840,0
